### Comparison of Covid-19 Infections and Deaths per 100K people, by State

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from IPython.display import display
import scrapbook as sb

# List of states to graph
states = ['Alaska', 'Connecticut', 'Colorado', 'Wisconsin', 'New York', 'South Dakota', 'Montana']

In [2]:
# Parameters that will be changed by the bmonreporter report-creating script.
# This cell must have a "parameters" tag.
server_web_address = None     # irrelevant, no BMON data used
org_id = None  # irrelevant, no BMON data used

In [3]:
# The report generator script needs to the know the Title of this report and where
# to sort it with respect to other reports.  Glue that information to this notebook.
sb.glue('title', 'COVID per 100K People, by State')
sb.glue('sort_order', 100)

In [4]:
dfs = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv', parse_dates=['date'])
dfs.set_index('date', inplace=True)

In [5]:
# get state populations into DataFrame
#df_pop = pd.read_csv('state-population.csv')
df_pop = pd.read_csv('https://raw.githubusercontent.com/alanmitchell/an-data/master/demographic/state-population.csv')
state_to_pop = dict(zip(df_pop.state, df_pop.population))

In [8]:
days_to_avg = 7
def new_per_capita_avg(df):
    try:
        pop_100k = state_to_pop[df.iloc[0].state] / 100000.
        new_cases = df.cases.diff() / pop_100k
        new_deaths = df.deaths.diff() / pop_100k
        new_deaths_p_100k_avg = new_deaths.rolling(days_to_avg).mean()
        new_cases_p_100k_avg = new_cases.rolling(days_to_avg).mean()
        return pd.DataFrame({'new_deaths_p_100k_avg': new_deaths_p_100k_avg, 'new_cases_p_100k_avg': new_cases_p_100k_avg})
    except:
        return pd.DataFrame({'new_deaths_p_100k_avg': [np.NaN]*len(df), 'new_cases_p_100k_avg': [np.NaN]*len(df)})

df_results = dfs.groupby('state').apply(new_per_capita_avg).reset_index()
df_results.dropna(inplace=True)

All values shown the graphs and tables below are 7-day rolling averages.

A selection of states are shown in the graphs below.
You can click on State names in the Legend to hide and then show again various states.
You can also click and drag a rectangle on the graph to zoom in on a particular region.

In [10]:
df_state_subset = df_results.query('state in @states')
fig = px.line(df_state_subset, x="date", y="new_cases_p_100k_avg", color='state')
fig.update_layout(
    yaxis_title_text = 'New Confirmed Cases per 100K People',
    yaxis_title_font_size = 18,
    xaxis_title_text = 'Date',
    xaxis_title_font_size = 20,
    title_text = 'New Confirmed Cases per 100,000 People',
    title_font_size = 30,
)
fig.show()

In [11]:
fig = px.line(df_state_subset, x="date", y="new_deaths_p_100k_avg", color='state')
fig.update_layout(
    yaxis_title_text = 'New Deaths per 100K People',
    yaxis_title_font_size = 20,
    xaxis_title_text = 'Date',
    xaxis_title_font_size = 20,
    title_text = 'New Deaths per 100,000 People',
    title_font_size = 30,
)
fig.show()

The tables below show new cases and new deaths per 100,000 people, sorted from the highest incidence states
to lowest.

In [12]:
days_to_avg = 7
print(f'Last Day of Data: {dfs.index[-1].strftime("%b %d, %Y")}')
print(f'Number of Days in Average: {days_to_avg}')
def new_per_capita(df):
    try:
        new_cases = df.cases.diff()
        new_deaths = df.deaths.diff()
        pop_100k = state_to_pop[df.iloc[0].state] / 100000.
        new_deaths_p_100k = new_deaths[-days_to_avg:].mean() / pop_100k
        new_cases_p_100k = new_cases[-days_to_avg:].mean() / pop_100k
        return pd.Series({'New Deaths per 100K People': new_deaths_p_100k, 'New Cases per 100K People': new_cases_p_100k})
    except:
        return pd.Series({'New Deaths per 100K People': np.NaN, 'New Cases per 100K People': np.NaN})

df_results2 = dfs.groupby('state').apply(new_per_capita)
df_results2.dropna(inplace=True)
display(df_results2.sort_values('New Cases per 100K People', ascending=False)[['New Cases per 100K People']])
df_results2.sort_values('New Deaths per 100K People', ascending=False)[['New Deaths per 100K People']]

Last Day of Data: Dec 02, 2020
Number of Days in Average: 7


,New Cases per 100K People
state,
South Dakota,97.874677
Minnesota,96.698305
Wyoming,92.463851
Nebraska,91.973506
Rhode Island,90.189140
New Mexico,87.622105
North Dakota,87.300733
Alaska,82.564982
Indiana,81.717909


,New Deaths per 100K People
state,
South Dakota,2.357648
North Dakota,1.480944
New Mexico,1.212716
Illinois,1.091285
Indiana,1.007946
Michigan,0.959833
Nebraska,0.945287
Iowa,0.932739
Wisconsin,0.863655


Developed by Alan Mitchell, alan@analysisnorth.com, using [data compiled by the New York Times](https://github.com/nytimes/covid-19-data).
This page is regenerated nightly using the most recent NY Times-compiled data.

Also see the NY Times [COVID US Tracking page](https://www.nytimes.com/interactive/2020/us/coronavirus-us-cases.html).